In [2]:
import pandas as pd
import yaml

# Knuth's X algorithm basics

Below are classes that implement elements out of which the sparse matrix used in Knuth's X algorithm will be build. The idea is pretty straightforward - each class is a quite literal impementation of waht his paper describes.

In [3]:
class node:

    def __init__(self):
        self.up = self
        self.down = self
        self.left = self
        self.right = self
        self.COL = None
        self.ROW = None

    def hideInCol(self):
        self.up.down = self.down
        self.down.up = self.up
        self.COL.row_count -= 1
        return self

    def showInCol(self):
        self.up.down = self
        self.down.up = self
        self.COL.row_count += 1
        return self

    def hideInRow(self):
        self.left.right = self.right
        self.right.left = self.left
        return self

    def showInRow(self):
        self.left.right = self
        self.right.left = self
        return self
    
    def deleteUnuseableRow(self):
        cur_cell = self.right 
        while cur_cell != self:
            print(cur_cell.COL.label)
            cur_cell.hideInCol()
            cur_cell = cur_cell.right
        print("done")
        self.hideInCol()
        return

    def hideWholeRow(self, start = None):
        if start == None: # if we just started the loop, don't hide me from my column, but go to my neighbor
            self.right.hideWholeRow(start = self)
            return
        if start != self: # we are not first and not last => hide me and continue
            self.hideInCol()
            self.right.hideWholeRow(start = start)
            return
        return # last possibility start == self => we have looped and can finish

    def showWholeRow(self, start = None):
        if start == None: # if we just started the loop, don't hide me from my column, but go to my neighbor
            self.left.showWholeRow(start = self)
            return
        if start != self: # we are not first and not last => hide me and continue
            self.showInCol()
            self.left.showWholeRow(start = start)
            return
        return # last possibility start == self => we have looped and can finish
    
    def getNeighbor(self, direction, by=0):
        neighbor = {
            'left'  : self.left,
            'right' : self.right,
            'up'    : self.up,
            'down'  : self.down
        }[direction]
        if by > 0:
            return neighbor.getNeighbor(direction, by = by-1)
        return neighbor

    def __repr__(self):

        return f"<root node>"



In [4]:
class columnHeader(node):

    def __init__(self, label):
        super().__init__()
        self.label = label
        self.row_count = 0
        self.COL = self

    def hideMe(self):
        self.hideInRow()
        cell = self.down
        while cell != self:
            cell.hideWholeRow()
            cell = cell.down
        return self

    def showMe(self):
        self.showInRow()
        cell = self.up
        while cell != self:
            cell.showWholeRow()
            cell = cell.up
        return self

    def appendRow(self, row):
        row.down = self
        row.up = self.up
        self.up.down = row
        self.up = row
        row.COL = self
        self.row_count += 1
        return self

    def addToRoot(self, root):
        root.appendColumn(self)
        return self

    def __repr__(self):

        return f"<(col) {self.label}>"

In [5]:
class rowHeader(node):
    
    def __init__(self, label):
        super().__init__()
        self.label = label
        self.ROW = self

    def appendColumn(self, col):
        col.right = self
        col.left = self.left
        self.left.right = col
        self.left = col
        col.ROW = self
        return self

    def addToRoot(self, root):
        root.appendRow(self)
        return self

    def __repr__(self):

        return f"<(row) {self.label}>"

In [6]:
class rootNode(columnHeader, rowHeader):

    def __init__(self):
        node.__init__(self)
        self.row_count = 0
        self.label = None
        self.ROW = self
        self.COL = self

    def addToRoot(self, root):
        pass

    def __repr__(self):

        return f"<(node) {self.label}>"

# Helper functions

In [7]:
def dict_to_tuples(dict):

    """
    dictionaries are sometimes cumbersone in addressing, so I will often transform them into a list of tuples
    """

    return [(k, v) for k, v in dict.items()]

In [8]:
def tuples_to_dicts(tuples):

    """
    inverse of the above
    """

    return { t[0]: t[1] for t in tuples}

In [9]:
def check_simple_criterion(personal_attributes, criterion_attributes):

    """Given two sets of attributes (each represented as a dictionary of (dimension name : dimension value) it checks if the attributes match. The idea is:
    - sparse_matrix_row_attr - is the target: the list of attributes to be checked
    - check_attr - is the soruce:  the conditions to check against
    then
    1. if any dim from conditions is missing in target, check cannot be perfomed and defaults to OK
    2. otherwise, for the common dimensions we check:
        a. if all values match - OK
        b. otherwise - FAIL
    """
    
    has_dims = {k[0] for k in personal_attributes}
    needs_dims = {k[0] for k in criterion_attributes}
    overlap_attrs = has_dims & needs_dims

    can_be_checked = overlap_attrs == criterion_attributes

    # not enough attributes present to perform the  check => default to OK
    if not can_be_checked:
        return True

    # check on each dimension OK => all OK
    target = tuples_to_dicts(personal_attributes)
    source = tuples_to_dicts(criterion_attributes)

    summary_check_result = { target[d] == source[d] for d in overlap_attrs } 
    

    return summary_check_result == {True} or summary_check_result == {False}



In [10]:
def check_set_of_criteria(personal_attributes, known_identity_criteria):

    """
    for a single target (sparse_matrix_row_attr) it applies the check across
    a set of different source conditions (check_set). Result is OK <=> each single check is OK.
    To see what 'source' and what 'target' mean in this context, check the description of check_simple_criterion()
    """

    summary_check_results = {
        check_simple_criterion(personal_attributes, item)
        for item in known_identity_criteria
    }
    
    return summary_check_results == {True}

# Let's go

## 1. read the riddle file

First we read in the YML fiel describing the riddle. For notes on how to cosntruct such a file, consult the readme.md in this folder

In [ ]:
riddle_file = "input_1_einstein.yml"

with open(riddle_file) as f:
    try:
        riddle_text= yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(exc)

## 2. reshape data

we then reshape the read in data into something easier to handle downstream. This is neither part of Knuth's algorithm, nor critical to how it was implemented here to solve Zebra Riddles. I jsut made a (poor?) design decision to have very copmpact input fil3s, but they need to be reshaped before use.

### 2.1 reshape "simple conditions"

This part cosntructs a list of "simple conditions".

A simple condition is one that is along the lines of

> Person who `Attrribute A` also is `Attrribute B`

encoded in the file as 

> ```yaml
> - ...
> - first_attribute_dimension_name : first_attribute_value
>   second_attribute_dimension_name : second_attribute_value
> - ...
> ```

Since each characteristic is alwasy comrpised of dimension (e.g. "house color" or "favourite drink") and the value on that dimension ("green" or "milk"), eventually each entry on the list (each "condition") is effectively strucuted as a dictionary:

```python
{
    "type" : "_is", # fixed value. redundant here, but kept to be easier to align with following data strucutres
    "params" : [ # here order does not matter
        (
            first_attribute_dimension_name,
            first_attribute_value 
        ),
        (
            second_attribute_dimension_name,
            second_attribute_value 
        )
    ]
}
```

i.e. "The `Spaniard` (`nationality`) owns the `dog` (`pet`)." will be encoded as 

> ```yaml
> - ...
> - nationality : Spanish
>   pet : dog
> - ...
> ```

and represented as a following data structure

```python
{
    "type" : "_is", 
    "params" : [ 
        ('nationality', 'Spanish'),
        ('pet',  'dog')
    ]
}

In [13]:
equalities_list = [ 
    {
        "type" : "_is",
        "params" : dict_to_tuples(is_condition)
    }
    for is_condition in riddle_text
       if len(is_condition) > 1
]

equalities_list

[{'type': '_is', 'params': [('nation', 'Norway'), ('_#', 1)]},
 {'type': '_is', 'params': [('nation', 'England'), ('house_color', 'red')]},
 {'type': '_is', 'params': [('nation', 'Dutch'), ('drink', 'tea')]},
 {'type': '_is', 'params': [('cig', 'cigars'), ('house_color', 'yellow')]},
 {'type': '_is', 'params': [('nation', 'Germany'), ('cig', 'pipe')]},
 {'type': '_is', 'params': [('_#', 3), ('drink', 'milk')]},
 {'type': '_is', 'params': [('cig', 'no_filter'), ('pet', 'birds')]},
 {'type': '_is', 'params': [('nation', 'Sweden'), ('pet', 'dogs')]},
 {'type': '_is', 'params': [('cig', 'menthol'), ('drink', 'beer')]},
 {'type': '_is', 'params': [('house_color', 'green'), ('drink', 'coffee')]}]

### 2.2 reshape "neighbor" conditions

this is the part where we untangle and reshape all condition in the form


> Person who `Attrribute A` lives in a house next to person who `Attrribute B`

The encoding is quite the same as above, with the difference that the `"type" : "_is"` fixed value is replaced by one denoting the neighborly relationship:

* `_left` - in case the person described by firt attribute on the list lives left (house number **less by 1**) of the person described by the second attribute
* `_left` - in case the person described by firt attribute on the list lives right (house number **increased by 1**) of the person described by the second attribute
* `_adj` (adjacent) - in case the two menioned persons live next to each other, but we don't knwo who lives on which side.

Example:

> The `green` [`colored`] house is immediately to *`the right`* of the house whoe owner `drink` `tea`.

would be encoded as 

> ```yaml
> - ...
> - _right:
>   - color: green
>   - drink: tea
> - ...
> ```

and represented as a following data structure

```python
{
    "type" : "_right", 
    "params" : [
        ('color', 'green'),
        ('drink', 'tea')
    ]
}
```

In [14]:
neighbors_list = [
    {
        "type" : condition_name,
        "params" : [
            attr
            for item in neighbor_condition[condition_name]
            for attr in dict_to_tuples(item)
        ]
    }
    for neighbor_condition in riddle_text
    for condition_name, condition_params in neighbor_condition.items() \
        if condition_name in ['_left', '_right', '_adj']
]

neighbors_list

[{'type': '_left',
  'params': [('house_color', 'green'), ('house_color', 'white')]},
 {'type': '_adj', 'params': [('cig', 'light'), ('pet', 'cat')]},
 {'type': '_adj', 'params': [('cig', 'light'), ('drink', 'water')]},
 {'type': '_adj', 'params': [('nation', 'Norway'), ('house_color', 'blue')]},
 {'type': '_adj', 'params': [('house_color', 'yellow'), ('pet', 'horses')]}]

### 2.3 reshape other data

The only other piece of information we can get is the attribute encoded in the question.

the data structure is the same, but we replace `"type" : "_is"` with  `"type" : "_question"`.

So the question

> Now, who `drinks` `water`? Who owns the [`pet`] `zebra`?

woudl be encoded as 

```yaml
- ...
- _question:
  - drink : water
  - pet : zebra
- ...
```

and represented as a following data structure

```python
{
    "type" : "_question", 
    "params" : [
        ('drink', 'water'),
        ('pet', 'zebra')
    ]
}
```

In [15]:
other_values = [
    {
        "type" : condition_name,
        "params" : [
            attr
            for item in neighbor_condition[condition_name]
            for attr in dict_to_tuples(item)
        ]
    }
    for neighbor_condition in riddle_text
    for condition_name, condition_params in neighbor_condition.items() \
        if condition_name in ['_question']
]

other_values

[{'type': '_question', 'params': [('pet', 'fish')]}]

### 2.4 collate all data

a list of all constraints to be used downstream
Once we compiled all known attributes described in the riddle, we cna cosntruct a dictionary of dimentions.

Each entry is the name of the dimension and a set of all it's possible values. An additional dimension named `_#` is added, that contains all possible house positions (1-based indexing)

In [16]:
all_constraints = equalities_list + neighbors_list + other_values

all_constraints

[{'type': '_is', 'params': [('nation', 'Norway'), ('_#', 1)]},
 {'type': '_is', 'params': [('nation', 'England'), ('house_color', 'red')]},
 {'type': '_is', 'params': [('nation', 'Dutch'), ('drink', 'tea')]},
 {'type': '_is', 'params': [('cig', 'cigars'), ('house_color', 'yellow')]},
 {'type': '_is', 'params': [('nation', 'Germany'), ('cig', 'pipe')]},
 {'type': '_is', 'params': [('_#', 3), ('drink', 'milk')]},
 {'type': '_is', 'params': [('cig', 'no_filter'), ('pet', 'birds')]},
 {'type': '_is', 'params': [('nation', 'Sweden'), ('pet', 'dogs')]},
 {'type': '_is', 'params': [('cig', 'menthol'), ('drink', 'beer')]},
 {'type': '_is', 'params': [('house_color', 'green'), ('drink', 'coffee')]},
 {'type': '_left',
  'params': [('house_color', 'green'), ('house_color', 'white')]},
 {'type': '_adj', 'params': [('cig', 'light'), ('pet', 'cat')]},
 {'type': '_adj', 'params': [('cig', 'light'), ('drink', 'water')]},
 {'type': '_adj', 'params': [('nation', 'Norway'), ('house_color', 'blue')]},
 {

In [96]:
class einstein_matrix:

    """
    This is a class to contain and manage all operations related to cosntructing and using the sparse matrix used in Knuth's X algorithm

    Attributes
    ----------
    constraints : list
        a list of all constraints defined in the riddle
    dicts : dictionary
        a dictionary of all dimensions implied by the constraints
    matrix_root : rootNode
        the root of the matrix, whera ll other nodes are (possibly indirectly) linked
     column_headers_dir : dictionary
        a directory of all column headers created for the matrix, indexed in a way
        that makes lookup easier the lookup happens when rows are linked to columns
        when the matrix is populated
    row_headers_dir : dictionary
        a directory of all row headers created for the matrix.

    Methods
    -------
    deduce_dimensions()
        deduces all riddle dimensions from known constraints
    create_row_labels()
        Constructs a list of persons that have viable attributes along with a text label for the matrix row.
    create_rows()
        Creates all rowHeader objects 
    """

    def __init__(self, list_of_constraints):

        self.constraints = list_of_constraints
        self.dicts = self.deduce_dimensions()
        self.matrix_root = rootNode()
        self.simple_row_headers_dir = []
        self.simple_column_headers_dir = {
            "_is" : {},
            "_first" : {},
            "_second" : {}
        }    

    def deduce_dimensions(self):

        """
        Transforms known cosntraints into a list of all possible dimensions (dimension names) with all possible values of each dimension
        """

        all_attrs = [p for constraint in self.constraints for p in constraint["params"] ]

        dicts = {i[0] : set() for i in all_attrs}
        [dicts[i[0]].add(i[1]) for i in all_attrs]
            
        max_items = max([len(dim) for dim in dicts.values()])
        # want _# to be first so
        del dicts["_#"]
        dicts = {"_#" : {r+1 for r in range(max_items)}} | dicts

        return dicts

    def create_row_labels(self):

        """
        Takes the dictionary of dimensions and known simple (not "neighbor") constraints and constructs a list of persons that have allowed attributes along with a text label for the matrix row. The cosntruction is iterative (dimension by dimension, value by value).
        As we iterate,a ny combination of attributes that is known to violate the indeintity cosntraints will be discarded 
        """
    
        labels = [[]]

        guards = [constraint["params"] for constraint in self.constraints if constraint["type"] == "_is"]

        for dim_name, dim_values in self.dicts.items():
            labels = [
                label + [(dim_name, value)]
                for label in labels 
                for value in dim_values
                  if check_set_of_criteria(
                    label + [(dim_name, value)],
                    guards
                )
            ]

        return labels

    def create_simple_rows(self):

        """
        Creates all rowHeader objects representing possible persons (combinations of attributes) and adds them to the matrix's root node and registers each row in the row directory for easier lookup
        """
        
        labels = self.create_row_labels()

        self.simple_row_headers_dir = [
            {
            "attrs" : label,
            "node" : rowHeader(
                    ":".join([
                        str(attr[1]) for attr in label
                    ])
                ).addToRoot(self.matrix_root)
            }
            for label in labels
        ]

        return self

    def create_complex_rows():

        """
        creates all the additional rows representing combined  neighbour pairs, for neighbour consstraints without specified direction, e.g. in the form "person X livex next to person Y"
        """
        
        adj_constraints = [c for c in self.constraints if c["type"] == "_adj"]
        number_of_houses = len(self.dicts.get("_#"))

        print(adj_constraints)

    def create_equality_columns(self):

        """
        creates all columnHEader objects for each "simple" - i.e. the identity - constraint
        """

        for dim_name, dim_values in self.dicts.items():

            for val in dim_values:

                label = f"{dim_name}:{val}"
                node = columnHeader(label).addToRoot(self.matrix_root)
                self.column_headers_dir["_is"][(dim_name, val)] = node

        return self

    def create_simple_neighbor_columns(self):

        """
        Creates columns for the "neighbor" constraints, i.e. ones in the form "Person with the attribute X lives next to person with attribure Y"
        """

        neighbors_constraints = [c for c in self.constraints if c["type"] in ['_left', '_right']]
        number_of_houses = len(self.dicts.get("_#"))

        for constr in neighbors_constraints:

            direction = constr["type"]
            first_attr = constr["params"][0]
            second_attr = constr["params"][1]

            dir_lbl = dir_label_map.get(direction, "?")

            if direction  == "_right":
                first_house = 1
                last_house = number_of_houses - 1
            else:
                first_house = 2
                last_house = number_of_houses
            
            first_dir = self.simple_column_headers_dir["_first"]
            first_dir[first_attr] = first_dir.get(first_attr, {})
            first_dir = first_dir[first_attr]

            second_dir = self.simple_column_headers_dir["_second"]
            second_dir[second_attr] = second_dir.get(second_attr, {})
            second_dir = second_dir[second_attr]

            for h in range(first_house, last_house + 1):
                
                if direction == "_right":
                    label_txt = f"{second_attr[0]}:{second_attr[1]} {dir_lbl} {first_attr[0]}:{first_attr[1]} ({h})"
                else:
                    label_txt = f"{first_attr[0]}:{first_attr[1]} {dir_lbl} {second_attr[0]}:{second_attr[1]} ({h})"
                
                node = columnHeader(label_txt).addToRoot(self.matrix_root)

                first_dir[h] = node
                second_dir[h] = node

        return self

In [97]:
em = einstein_matrix(all_constraints)
em.deduce_dimensions()
em.create_simple_rows()
em.simple_row_headers_dir

[{'attrs': [('_#', 1),
   ('nation', 'Germany'),
   ('house_color', 'white'),
   ('drink', 'coffee'),
   ('cig', 'no_filter'),
   ('pet', 'dogs')],
  'node': <(row) 1:Germany:white:coffee:no_filter:dogs>},
 {'attrs': [('_#', 1),
   ('nation', 'Germany'),
   ('house_color', 'white'),
   ('drink', 'coffee'),
   ('cig', 'no_filter'),
   ('pet', 'horses')],
  'node': <(row) 1:Germany:white:coffee:no_filter:horses>},
 {'attrs': [('_#', 1),
   ('nation', 'Germany'),
   ('house_color', 'white'),
   ('drink', 'coffee'),
   ('cig', 'no_filter'),
   ('pet', 'fish')],
  'node': <(row) 1:Germany:white:coffee:no_filter:fish>},
 {'attrs': [('_#', 1),
   ('nation', 'Germany'),
   ('house_color', 'white'),
   ('drink', 'coffee'),
   ('cig', 'no_filter'),
   ('pet', 'cat')],
  'node': <(row) 1:Germany:white:coffee:no_filter:cat>},
 {'attrs': [('_#', 1),
   ('nation', 'Germany'),
   ('house_color', 'white'),
   ('drink', 'coffee'),
   ('cig', 'no_filter'),
   ('pet', 'birds')],
  'node': <(row) 1:German

In [73]:
def search(matrix, solution=[], log=[], k=0):

    if type(matrix) != rootNode:
        raise TypeError("Must start with matrix root node")

    # solution found if matrix is empty
    if matrix.right == matrix:
        print("ready")
        return prep_solution(solution)

    # select column
    col = matrix.right
    tmp_col = col
    while tmp_col != matrix:
        if tmp_col.row_count < col.row_count:
            col = tmp_col
        tmp_col = tmp_col.right
    print(col.label)

    # hide column 
    col.hideMe()

    # go through its each row
    row = col.down
    print(row.ROW.label)
    while row != col:

        # add row to solution
        solution.append(row)
        log.append(f"select R{row.ROW.label['R']}C{row.ROW.label['C']}={row.ROW.label['#']}")

        # hide row's columns
        cell = row.right
        while cell != row:
            if type(cell) != rowHeader:
                cell.COL.hideMe()
            cell = cell.right

        # go deeper
        result = search(matrix, solution, log, k = k+1)
        if result != None:
            return result

        # uncover columns and remove row from solution
        solution.pop()
        log.append("back")
        cell = row.left
        while cell != row:
            if type(cell) != rowHeader:
                cell.COL.showMe()
            cell = cell.left

        # next row
        row = row.down

    # uncover column
    col.showMe()
    
    return None

    